In [14]:
import PyPDF2

text_content = []

def extract_text(pdf_file):
    # Open the pdf file
    with open(pdf_file, 'rb') as file:
        # Initialize a pdf file reader
        reader = PyPDF2.PdfReader(file)

        # Loop through each page
        for i in range(len(reader.pages)):
            # Get the text content of the page
            page = reader.pages[i]
            text = page.extract_text()

            # Save the text to a separate file

            # Add the text to the list
            text_content.append(text)
extract_text(r'C:\Users\Sergio\Desktop\tws-workspace\closeAI\Examples\content_generation\content\Introduction to Human Nutrition 2nd Edition ( PDFDrive ).pdf')

In [17]:
print(text_content[3])

Manual de Nutrición  y Dietética 
Ángeles Carbajal Azcona. Departamento  de Nutrición.  Facultad de Farmacia.  Universidad  Complutense  de Madrid 
https://www.ucm.es/nutricioncarbajal/                                                                                                                                         4 
 
 o Carnes y derivados  
o Pescados  y mariscos 
 Bebidas 
 Condimentos  y especias 
 Pérdidas de vitaminas  al procesar los alimentos  
 Fortificación  o enriquecimiento  de los alimentos  
 Productos  light o ligeros 
 Etiquetado  nutricional  
15. Valoración  del estado nutricional ………………………………………………………  
 Encuestas  dietéticas  
 Diario dietético 
 Recuerdo  de 24 horas 
 Historia dietética 
 Modelos de cuestionarios  151
16. Dieta equilibrada  …………………………………………………………………………………
 Concepto  actual de dieta prudente  y nutrición óptima 
 Características  de la dieta equilibrada  
 Beneficios  de componentes  no nutritivos  
 Recomendaciones  dietéticas